<a href="https://colab.research.google.com/github/SarahLares/Coursera_Capstone/blob/master/Clustering_Neighborhoods_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Segmenting and Clustering Neighborhoods in Toronto


##Part 1: Preprocessing of Toronto Data

### Import Libraries

In [21]:
import numpy as np 
import pandas as pd

### Importing the Data as DataFrame

In [22]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url)
df_toronto = pd.read_json(df[0].to_json(orient='records'))

### Delete the intances whit **Borough = Not assigned**

In [23]:
df_toronto = df_toronto.loc[df_toronto['Borough']!='Not assigned'].reset_index(drop=True)

In [24]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


We can see that the data have grouped the Neighbourhood whit the same Postal Code

In [25]:
df_toronto.loc[df_toronto['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


The data don't have **Neighbourhood = Not assigned**

### Shape of the data

In [26]:
df_toronto.shape

(103, 3)

## Part 2: Get Latitude and Longitude for Neighbourhood

In [27]:
url_geo = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url_geo)

In [46]:
df_toronto = df_toronto.merge(df_geo, left_on='Postal Code', right_on='Postal Code')

In [47]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
